# NYC Crime Data Analysis

## About the data

It visualizes the trends of major crimes in the New York across time and locations. I'm interested in whether specific time during a day, or specific day during a week, or specific month in a year would have more crime instances than other of the time periods. I want to explore with the dataset and predict the probabilty of crime categories. The datasets are provided by NYC Open Data. I used [NYPD_Complaint_Data_Current dataset](https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Current-YTD/5uac-w243 ) for 2017 and NYPD_Complaint_Data_Historic for previous years. The dataset includes information on the type of crime, the location where the arrest occurred, and the date/time. There are 3 different types of law_category as VIOLATION, MISDEMEANOR, FELONY and there are offense categories such as PETIT LARCENY, HARRASSMENT, ASSAULT, DANGEROUS DRUGS, GRAND LARCENY.  
  
The [NYPD_Complaint_Data_Historic](https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i) dataset is too large to fit into the memory and we can't work with it in pandas. We sampled the data and the new file ( NYPD_Complaint_Data_Historic_sample.csv ) is 10% of the orginal dataset. We randomly select 558003 lines from the suffeled historic data (the file contains 5580036 lines). The probability of being selected be the same for every line in the file.

**Field Names and Descriptions are as follows:**

In [1]:
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
from plotly.tools import FigureFactory as ff
init_notebook_mode(connected=True)

dc = { "Column":["CMPLNT_NUM", "CMPLNT_FR_DT", "CMPLNT_FR_TM", "CMPLNT_TO_DT", "CMPLNT_TO_TM", "RPT_DT", "KY_CD", "OFNS_DESC", "PD_CD", "PD_DESC", "CRM_ATPT_CPTD_CD", "LAW_CAT_CD", "JURIS_DESC", "BORO_NM", "ADDR_PCT_CD", "LOC_OF_OCCUR_DESC", "PREM_TYP_DESC", "PARKS_NM", "HADEVELOPT", "X_COORD_CD", "Y_COORD_CD", "Latitude", "Longitude"],
       "Description":["Randomly generated persistent ID for each complaint ", "Exact date of occurrence for the reported event (or starting date of occurrence, if CMPLNT_TO_DT exists)", "Exact time of occurrence for the reported event (or starting time of occurrence, if CMPLNT_TO_TM exists)", "Ending date of occurrence for the reported event, if exact time of occurrence is unknown", "Ending time of occurrence for the reported event, if exact time of occurrence is unknown", "Date event was reported to police ", "Three digit offense classification code", "Description of offense corresponding with key code", "Three digit internal classification code (more granular than Key Code)", "Description of internal classification corresponding with PD code (more granular than Offense Description)", "Indicator of whether crime was successfully completed or attempted, but failed or was interrupted prematurely", "Level of offense: felony, misdemeanor, violation ", "Jurisdiction responsible for incident. Either internal, like Police, Transit, and Housing; or external, like Correction, Port Authority, etc.", "The name of the borough in which the incident occurred", "The precinct in which the incident occurred", "Specific location of occurrence in or around the premises; inside, opposite of, front of, rear of", "Specific description of premises; grocery store, residence, street, etc.", "Name of NYC park, playground or greenspace of occurrence, if applicable (state parks are not included)", "Name of NYCHA housing development of occurrence, if applicable", "X-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104)", "Y-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104)", "Latitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326) ", "Longitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326)"]  
       }
dc_df=pd.DataFrame.from_dict(dc)

table = ff.create_table(dc_df)
iplot(table, filename='simple_table')

## Methodology

Do more violent crime happen in New York other crime incidents?  
Are there any relationships between specific months/days/time of the year?  
Do more crime incidents happen in certain geographic areas of the city?  

**Our hypothesis:**  
What is the proportion of types of crime those have been committed in the different parts of the city.  
Let's assume that the violance related crime has a highest probaility.  

We test the prediction by looking the pattern of crime and the known data if we find evidence our hypothesis has to be consider true.  

**Analysis plan:**  
We load and clean the dataset.  
Sample and clean the historical data file.  
Explore the data and find pattern and relationship between the features.  
Creating new features if necessary.  
Build predictive models and choose the best model for forecast.  
Evalute the performance of the model  
Conclusion  

## Setup

In [2]:
import numpy as np
import pandas as pd

from IPython.display import display

import matplotlib  
import matplotlib.pyplot as plt  
from matplotlib import rcParams

import seaborn as sns
sns.set(style='ticks', palette='Set2')

from altair import Chart, Color, Scale

import calendar

In [3]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [4]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [5]:
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 9999)

## Dataset for 2017

## Dataset

In [6]:
# Retrieving Data

df=pd.read_csv('NYPD_Complaint_Data_Current_YTD.csv')
df.head(1)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,JURIS_DESC,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,845348933,03/31/2017,23:30:00,NaN,NaN,03/31/2017,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,N.Y. POLICE DEPT,BROOKLYN,69.0,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,"1,012,423","171,737",40.638018,-73.898491,"(40.638018389, -73.898491201)"


In [7]:
# rename the columns

from collections import OrderedDict

columns = OrderedDict()
columns['CMPLNT_FR_DT'] = "from_date" 
columns['CMPLNT_FR_TM'] = "from_time" 
columns['CMPLNT_TO_DT'] = "to_date" 
columns['CMPLNT_TO_TM'] = "to_time" 
columns['OFNS_DESC'] = "offense" 
columns['PD_DESC'] = "offense_description" 
columns['LAW_CAT_CD'] = "law_category" 
columns['LOC_OF_OCCUR_DESC'] = "occurence" 
columns['PREM_TYP_DESC'] = "premise" 
columns['BORO_NM'] = "borough" 
columns['Latitude'] = "latitude" 
columns['Longitude'] = "longitude"

df = df[list(columns.keys())]
df.columns = columns.values()

df.dropna(subset=['latitude','longitude'], inplace=True)

df.head(1)

,from_date,from_time,to_date,to_time,offense,offense_description,law_category,occurence,premise,borough,latitude,longitude
0,03/31/2017,23:30:00,NaN,NaN,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",VIOLATION,INSIDE,RESIDENCE - APT. HOUSE,BROOKLYN,40.638018,-73.898491


## Data cleaning

In [8]:
# no. of the rows and columns

df.shape

(106466, 12)

In [9]:
# checking missing values

print(df.isnull().sum())

from_date                  0
from_time                  0
to_date                16786
to_time                16748
offense                   13
offense_description       60
law_category               0
occurence              20061
premise                  452
borough                    0
latitude                   0
longitude                  0
dtype: int64


In [10]:
# correct incorrect date

df.loc[df['from_date'].str.contains('1017'), 'from_date'] = df['from_date'].str.replace('1017', '2017')

In [11]:
# convert to datetime

df['from_date']=pd.to_datetime(df['from_date'])
df['to_date']=pd.to_datetime(df['to_date'])

In [12]:
df.dtypes

from_date              datetime64[ns]
from_time                      object
to_date                datetime64[ns]
to_time                        object
offense                        object
offense_description            object
law_category                   object
occurence                      object
premise                        object
borough                        object
latitude                      float64
longitude                     float64
dtype: object

In [13]:
# add new coloums

# year

df['year'] = pd.DatetimeIndex(df['from_date']).year


# Month

df['Month'] = df['from_date'].dt.month
df['Month'] = df['Month'].apply(lambda x: calendar.month_abbr[x])

# Weekday

df['weekday'] = df['from_date'].dt.weekday_name


In [14]:
# Holiday

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

dr = pd.date_range(start='2005-01-01', end='2017-05-01')
dff = pd.DataFrame()
dff['Date'] = dr

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

dff['Holiday'] = dff['Date'].isin(holidays)
dff.columns = ['from_date', 'holiday']
dff.sample(10)

,from_date,holiday
2411,2011-08-09,False
844,2007-04-25,False
4049,2016-02-02,False
1665,2009-07-24,False
1597,2009-05-17,False
3361,2014-03-16,False
3986,2015-12-01,False
527,2006-06-12,False
2210,2011-01-20,False
2725,2012-06-18,False


In [15]:
df=pd.merge(df, dff, how='left', on='from_date')

In [16]:
# hour

df['hour'] = pd.DatetimeIndex(df['from_time']).hour

In [17]:
# create time column
from datetime import datetime

df['time_str'] = df.from_date.astype(str) + " " + df.from_time.astype(str)
df['time'] = df.time_str.map(lambda v:datetime.strptime(v, '%Y-%m-%d %H:%M:%S'))

In [18]:
# no. of the month
df['no_of_month'] = df.time.dt.month

In [19]:
# Day of the month
df['Day_of_month'] = df['from_date'].dt.day

In [20]:
# column no. of the weekday
import calendar

d=dict(enumerate(calendar.day_name))

d_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
d_df.columns = ['no_of_weekday', 'weekday']

df=pd.merge(df, d_df, how='left', on='weekday')

In [21]:
df.sample(1)

,from_date,from_time,to_date,to_time,offense,offense_description,law_category,occurence,premise,borough,latitude,longitude,year,Month,weekday,holiday,hour,time_str,time,no_of_month,Day_of_month,no_of_weekday
57953,2017-02-08,12:00:00,2017-02-08,14:00:00,OFF. AGNST PUB ORD SENSBLTY &,AGGRAVATED HARASSMENT 2,MISDEMEANOR,FRONT OF,RESIDENCE - APT. HOUSE,QUEENS,40.746473,-73.876261,2017,Feb,Wednesday,False,12,2017-02-08 12:00:00,2017-02-08 12:00:00,2,8,2


In [22]:
# unique values in column year
df.year.unique()

array([2017, 2016, 2015, 2013, 2012, 2010, 2014, 2011, 2009, 2006, 2008,
       2005, 2007, 2003, 1996, 1997, 1998, 1983, 2004, 2001, 1916, 1977,
       2000, 2002, 1984, 1986])

In [23]:
print('2006: ',(len(df[df.year == 2006])))
print('2015: ',(len(df[df.year == 2015])))
print('2016: ',(len(df[df.year == 2016])))
print('2017: ',(len(df[df.year == 2017])))

2006:  3
2015:  261
2016:  5568
2017:  100361


In [24]:
# drop lines before 2016

df_2017=df.copy()
df_2017= df_2017[df_2017.year > 2016]
df_2017.year.unique()

array([2017])

In [25]:
# checking date

print('min of from date', df_2017.from_date.min())
print('max of from date', df_2017.from_date.max())
print('min of to date', df_2017.to_date.min())
print('max of to date', df_2017.to_date.max())

min of from date 2017-01-01 00:00:00
max of from date 2017-12-14 00:00:00
min of to date 2016-02-27 00:00:00
max of to date 2017-11-28 00:00:00


In [26]:
# drop lines after 2017-03-31 00:00:00

display(df_2017.ix[df_2017['from_date'] > '2017-03-31 00:00:00'])
display(df_2017.ix[df_2017['to_date'] > '2017-03-31 00:00:00'])

/Users/bernadett/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



,from_date,from_time,to_date,to_time,offense,offense_description,law_category,occurence,premise,borough,latitude,longitude,year,Month,weekday,holiday,hour,time_str,time,no_of_month,Day_of_month,no_of_weekday
96361,2017-12-14,23:30:00,2016-12-20,23:45:00,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",FELONY,NaN,STREET,BROOKLYN,40.695916,-73.983218,2017,Dec,Thursday,NaN,23,2017-12-14 23:30:00,2017-12-14 23:30:00,12,14,3


,from_date,from_time,to_date,to_time,offense,offense_description,law_category,occurence,premise,borough,latitude,longitude,year,Month,weekday,holiday,hour,time_str,time,no_of_month,Day_of_month,no_of_weekday
73144,2017-01-27,22:00:00,2017-11-28,NaN,CRIMINAL MISCHIEF & RELATED OF,"MISCHIEF,CRIMINAL, UNCL 2ND",FELONY,FRONT OF,RESIDENCE - APT. HOUSE,BROOKLYN,40.617969,-73.978966,2017,Jan,Friday,False,22,2017-01-27 22:00:00,2017-01-27 22:00:00,1,27,4


In [27]:
df_2017 = df_2017.drop(df_2017[df_2017.from_date > '2017-03-31 00:00:00'].index)
df_2017 = df_2017.drop(df_2017[df_2017.to_date > '2017-03-31 00:00:00'].index)

In [28]:
print(df_2017.from_date.max())
print(df_2017.to_date.max())

2017-03-31 00:00:00
2017-03-31 00:00:00


In [29]:
# checking mistyping

diff=df_2017['to_date'] - df_2017['from_date']
print('min: ',diff.min())
print('max: ',diff.max())

min:  -365 days +00:00:00
max:  89 days 00:00:00


In [30]:
display(df_2017.loc[diff == '-365 days +00:00:00'])

,from_date,from_time,to_date,to_time,offense,offense_description,law_category,occurence,premise,borough,latitude,longitude,year,Month,weekday,holiday,hour,time_str,time,no_of_month,Day_of_month,no_of_weekday
38213,2017-02-26,14:30:00,2016-02-27,NaN,CRIMINAL MISCHIEF & RELATED OF,"CRIMINAL MISCHIEF,UNCLASSIFIED 4",MISDEMEANOR,FRONT OF,STREET,BRONX,40.85394,-73.892439,2017,Feb,Sunday,False,14,2017-02-26 14:30:00,2017-02-26 14:30:00,2,26,6


In [31]:
# checking difference is negative (<0) as we can have data for year 2017

def to_int(v):
    try:
        return int(str(v).split()[0])
    except:
        return 0

df_diff = pd.DataFrame(diff, columns=['diff'])
df_diff['days'] = df_diff['diff'].map(to_int)
df_diff[df_diff.days < 0]

,diff,days
38213,-365 days,-365


In [32]:
# drop line where the difference = - 365 days

df_2017 = df_2017.drop(df_2017[diff == '-365 days +00:00:00'].index)

In [33]:
# Identifying missing values in offense coloumn

print(df_2017['offense'].isnull().sum())
display(df_2017[df_2017['offense'].isnull()])

13


,from_date,from_time,to_date,to_time,offense,offense_description,law_category,occurence,premise,borough,latitude,longitude,year,Month,weekday,holiday,hour,time_str,time,no_of_month,Day_of_month,no_of_weekday
37605,2017-02-27,17:05:00,NaT,NaN,NaN,TAX LAW,FELONY,INSIDE,CANDY STORE,STATEN ISLAND,40.514921,-74.245439,2017,Feb,Monday,False,17,2017-02-27 17:05:00,2017-02-27 17:05:00,2,27,0
40702,2017-02-24,23:00:00,2017-02-24,23:15:00,NaN,TAX LAW,FELONY,INSIDE,GROCERY/BODEGA,BROOKLYN,40.696082,-73.943160,2017,Feb,Friday,False,23,2017-02-24 23:00:00,2017-02-24 23:00:00,2,24,4
42161,2017-02-23,18:09:00,2017-02-23,18:30:00,NaN,TAX LAW,FELONY,INSIDE,GROCERY/BODEGA,STATEN ISLAND,40.572953,-74.118360,2017,Feb,Thursday,False,18,2017-02-23 18:09:00,2017-02-23 18:09:00,2,23,3
42162,2017-02-23,18:09:00,2017-02-23,18:30:00,NaN,TAX LAW,FELONY,INSIDE,GROCERY/BODEGA,STATEN ISLAND,40.572953,-74.118360,2017,Feb,Thursday,False,18,2017-02-23 18:09:00,2017-02-23 18:09:00,2,23,3
44505,2017-02-21,19:37:00,2017-02-21,21:25:00,NaN,TAX LAW,FELONY,INSIDE,GROCERY/BODEGA,BROOKLYN,40.585640,-73.987609,2017,Feb,Tuesday,False,19,2017-02-21 19:37:00,2017-02-21 19:37:00,2,21,1
47775,2017-02-18,21:10:00,2017-02-18,22:00:00,NaN,TAX LAW,FELONY,INSIDE,GROCERY/BODEGA,BROOKLYN,40.697633,-73.929469,2017,Feb,Saturday,False,21,2017-02-18 21:10:00,2017-02-18 21:10:00,2,18,5
69313,2017-01-31,10:00:00,2017-01-31,10:30:00,NaN,TAX LAW,FELONY,INSIDE,GROCERY/BODEGA,BRONX,40.834272,-73.917110,2017,Jan,Tuesday,False,10,2017-01-31 10:00:00,2017-01-31 10:00:00,1,31,1
73520,2017-01-27,20:20:00,2017-01-27,20:26:00,NaN,TAX LAW,FELONY,INSIDE,RESIDENCE - PUBLIC HOUSING,BROOKLYN,40.718454,-73.937106,2017,Jan,Friday,False,20,2017-01-27 20:20:00,2017-01-27 20:20:00,1,27,4
77663,2017-01-24,19:30:00,2017-01-24,20:15:00,NaN,TAX LAW,FELONY,INSIDE,SMALL MERCHANT,STATEN ISLAND,40.524912,-74.200635,2017,Jan,Tuesday,False,19,2017-01-24 19:30:00,2017-01-24 19:30:00,1,24,1
82544,2017-01-20,20:55:00,NaT,NaN,NaN,TAX LAW,FELONY,INSIDE,FAST FOOD,BROOKLYN,40.658549,-73.981998,2017,Jan,Friday,False,20,2017-01-20 20:55:00,2017-01-20 20:55:00,1,20,4


In [34]:
df_2017.shape

(100358, 22)

In [35]:
# show unique values of offense if the offense_description='TAX LAW'
df_2017[df_2017.offense_description=='TAX LAW'].offense.unique()

array(['OTHER STATE LAWS (NON PENAL LA', nan], dtype=object)

In [36]:
# drop missing values offense column

df_2017 = df_2017[(pd.notnull(df_2017['offense']))]
df_2017['offense'].isnull().sum()

0

In [37]:
df_2017.shape

(100345, 22)

In [38]:
# replace True False with 1 and 0 in 'holiday' coloumn

df_2017.holiday = df_2017.holiday.astype(int)
df_2017.holiday.unique()

array([0, 1])

In [39]:
# checking no. of offense type, law category, borough, year

print('no. of offense: ',len(pd.unique(df_2017.offense)))
print('no. of law category: ',len(pd.unique(df_2017.law_category)))
print('no. of borough: ',len(pd.unique(df_2017.borough)))
print('no. of year: ',len(pd.unique(df_2017.year)))

no. of offense:  55
no. of law category:  3
no. of borough:  5
no. of year:  1


## Historical dataset

## Dataset

In [40]:
# Retrieving Data

df_his=pd.read_csv('NYPD_Complaint_Data_Historic_sample.csv', header=None)
df_his.columns = ['CMPLNT_NUM','CMPLNT_FR_DT','CMPLNT_FR_TM','CMPLNT_TO_DT','CMPLNT_TO_TM','RPT_DT','KY_CD','OFNS_DESC','PD_CD','PD_DESC','CRM_ATPT_CPTD_CD','LAW_CAT_CD','JURIS_DESC','BORO_NM','ADDR_PCT_CD','LOC_OF_OCCUR_DESC','PREM_TYP_DESC','PARKS_NM','HADEVELOPT','X_COORD_CD','Y_COORD_CD','Latitude','Longitude','Lat_Lon']
df_his.head(1)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,JURIS_DESC,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,889128095,08/27/2014,08:05:00,08/27/2014,15:00:00,08/27/2014,109,GRAND LARCENY,438.0,"LARCENY,GRAND FROM BUILDING (NON-RESIDENCE) UN...",COMPLETED,FELONY,N.Y. POLICE DEPT,MANHATTAN,5.0,INSIDE,PRIVATE/PAROCHIAL SCHOOL,NaN,NaN,"984,179","202,071",40.721323,-74.000258,"(40.721323154, -74.000257587)"


In [41]:
# rename the columns

from collections import OrderedDict

columns = OrderedDict()
columns['CMPLNT_FR_DT'] = "from_date" 
columns['CMPLNT_FR_TM'] = "from_time" 
columns['CMPLNT_TO_DT'] = "to_date" 
columns['CMPLNT_TO_TM'] = "to_time" 
columns['OFNS_DESC'] = "offense" 
columns['PD_DESC'] = "offense_description" 
columns['LAW_CAT_CD'] = "law_category" 
columns['LOC_OF_OCCUR_DESC'] = "occurence" 
columns['PREM_TYP_DESC'] = "premise" 
columns['BORO_NM'] = "borough" 
columns['Latitude'] = "latitude" 
columns['Longitude'] = "longitude"

df_his = df_his[list(columns.keys())]
df_his.columns = columns.values()

df_his.dropna(subset=['latitude','longitude'], inplace=True)

df_his.head(1)

,from_date,from_time,to_date,to_time,offense,offense_description,law_category,occurence,premise,borough,latitude,longitude
0,08/27/2014,08:05:00,08/27/2014,15:00:00,GRAND LARCENY,"LARCENY,GRAND FROM BUILDING (NON-RESIDENCE) UN...",FELONY,INSIDE,PRIVATE/PAROCHIAL SCHOOL,MANHATTAN,40.721323,-74.000258


## Data cleaning

In [42]:
# no. of the rows and columns

df_his.shape

(538319, 12)

In [43]:
# checking missing values

print(df_his.isnull().sum())

from_date                  73
from_time                   4
to_date                141300
to_time                140921
offense                  1757
offense_description       483
law_category                0
occurence              114376
premise                  3409
borough                     1
latitude                    0
longitude                   0
dtype: int64


In [44]:
# drop missing from_date and from_time rows

df_his = df_his[(pd.notnull(df_his['from_date'])) & (pd.notnull(df_his['from_time']))]

In [45]:
print(df_his.isnull().sum())

from_date                   0
from_time                   0
to_date                141290
to_time                140911
offense                  1757
offense_description       483
law_category                0
occurence              114359
premise                  3409
borough                     1
latitude                    0
longitude                   0
dtype: int64


In [46]:
# correct incorrect date

df_his.loc[df_his['from_date'].str.contains('1015'), 'from_date'] = df_his['from_date'].str.replace('1015', '2015')

In [47]:
# convert to datetime

df_his['from_date']=pd.to_datetime(df_his['from_date'])
df_his['to_date']=pd.to_datetime(df_his['to_date'])

In [48]:
df_his.dtypes

from_date              datetime64[ns]
from_time                      object
to_date                datetime64[ns]
to_time                        object
offense                        object
offense_description            object
law_category                   object
occurence                      object
premise                        object
borough                        object
latitude                      float64
longitude                     float64
dtype: object

In [49]:
# add new coloums

# year

df_his['year'] = pd.DatetimeIndex(df_his['from_date']).year


# Month

df_his['Month'] = df_his['from_date'].dt.month
df_his['Month'] = df_his['Month'].apply(lambda x: calendar.month_abbr[x])

# Weekday

df_his['weekday'] = df_his['from_date'].dt.weekday_name


In [50]:
# unique values in column year

df_his.year.unique()

array([2014, 2011, 2010, 2007, 2016, 2015, 2012, 2013, 2006, 2008, 2009,
       2005, 1986, 2000, 2004, 2003, 1997, 1996, 2001, 2002, 1999, 1995,
       1991, 1959, 1908, 1988, 1994, 1968, 1979, 1998, 1992, 1914, 1966,
       1965, 1985, 1913, 1962, 1960, 1976, 1984, 1993, 1990, 1987, 1974,
       1980, 1970, 1941, 1989, 1915, 1967, 1912])

In [51]:
print('Y 2005:',len(df_his[df_his.year==2005]))
print('Y 2006:',len(df_his[df_his.year==2006]))

Y 2005: 976
Y 2006: 51580


In [52]:
# keep rows where the year > 2005

df_his= df_his[df_his.year > 2005]
df_his.year.unique()

array([2014, 2011, 2010, 2007, 2016, 2015, 2012, 2013, 2006, 2008, 2009])

In [53]:
# Holiday
df_his=pd.merge(df_his, dff, how='left', on='from_date')

In [54]:
# replace True False with 1 and 0 in 'holiday' coloumn

df_his.holiday = df_his.holiday.astype(int)
df_his.holiday.unique()

array([0, 1])

In [55]:
# checking hour in from_time column
df_his['from_time'].str[:2].unique()

array(['08', '14', '18', '17', '13', '15', '21', '20', '19', '01', '23',
       '00', '10', '02', '16', '07', '12', '22', '03', '09', '11', '04',
       '05', '06', '24'], dtype=object)

In [56]:
# convert '24' hour to '00' hour in 'from_time' column
from datetime import datetime

def fix_time(v):
    h,m,s = str(v).split(":")
    if h=="24":
        h = "00"
    return ":".join([h,m,s])

df_his['from_time'] = df_his['from_time'].map(fix_time)
df_his.sample(1)

,from_date,from_time,to_date,to_time,offense,offense_description,law_category,occurence,premise,borough,latitude,longitude,year,Month,weekday,holiday
197488,2008-11-12,09:40:00,2008-11-12,09:50:00,HARRASSMENT 2,"HARASSMENT,SUBD 1,CIVILIAN",VIOLATION,FRONT OF,STREET,BROOKLYN,40.606587,-73.983273,2008,Nov,Wednesday,0


In [57]:
df_his['from_time'].str[:2].unique()

array(['08', '14', '18', '17', '13', '15', '21', '20', '19', '01', '23',
       '00', '10', '02', '16', '07', '12', '22', '03', '09', '11', '04',
       '05', '06'], dtype=object)

In [58]:
# hour

df_his['hour'] = pd.DatetimeIndex(df_his['from_time']).hour

In [59]:
# create time column
from datetime import datetime

df_his['time_str'] = df_his.from_date.astype(str) + " " + df_his.from_time.astype(str)
df_his['time'] = df_his.time_str.map(lambda v:datetime.strptime(v, '%Y-%m-%d %H:%M:%S'))

In [60]:
# no. of the month
df_his['no_of_month'] = df_his.time.dt.month

In [61]:
# Day of the month
df_his['Day_of_month'] = df_his['from_date'].dt.day

In [62]:
# column no. of the weekday

df_his=pd.merge(df_his, d_df, how='left', on='weekday')

In [63]:
df_his.sample(1)

,from_date,from_time,to_date,to_time,offense,offense_description,law_category,occurence,premise,borough,latitude,longitude,year,Month,weekday,holiday,hour,time_str,time,no_of_month,Day_of_month,no_of_weekday
397378,2013-06-06,01:00:00,NaT,NaN,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",FELONY,INSIDE,RESIDENCE - APT. HOUSE,BRONX,40.838551,-73.909399,2013,Jun,Thursday,0,1,2013-06-06 01:00:00,2013-06-06 01:00:00,6,6,3


In [64]:
# checking date

print('min of from date', df_his.from_date.min())
print('max of from date', df_his.from_date.max())
print('min of to date', df_his.to_date.min())
print('max of to date', df_his.to_date.max())

min of from date 2006-01-01 00:00:00
max of from date 2016-12-31 00:00:00
min of to date 2006-01-01 00:00:00
max of to date 2017-01-04 00:00:00


In [65]:
df_his.year.unique()

array([2014, 2011, 2010, 2007, 2016, 2015, 2012, 2013, 2006, 2008, 2009])

In [66]:
# checking mistyping

diff_h=df_his['to_date'] - df_his['from_date']
print('min: ',diff_h.min())
print('max: ',diff_h.max())

min:  -366 days +00:00:00
max:  3653 days 00:00:00


In [67]:
# checking difference is negative (<0) 

def to_int(v):
    try:
        return int(str(v).split()[0])
    except:
        return 0

df_diff_h = pd.DataFrame(diff_h, columns=['diff_h'])
df_diff_h['days'] = df_diff_h['diff_h'].map(to_int)
df_diff_h[df_diff_h.days < 0]

,diff_h,days
141205,-366 days,-366
252659,-365 days,-365
335103,-153 days,-153
355423,-1 days,-1


In [68]:
display(df_his.loc[(diff_h == '-366 days +00:00:00') | (diff_h == '-365 days +00:00:00')|(diff_h == '-153 days +00:00:00')| (diff_h == '-1 days +00:00:00')])

,from_date,from_time,to_date,to_time,offense,offense_description,law_category,occurence,premise,borough,latitude,longitude,year,Month,weekday,holiday,hour,time_str,time,no_of_month,Day_of_month,no_of_weekday
141205,2012-04-07,12:15:00,2011-04-07,NaN,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR,INSIDE,RESIDENCE - APT. HOUSE,QUEENS,40.581846,-73.830892,2012,Apr,Saturday,0,12,2012-04-07 12:15:00,2012-04-07 12:15:00,4,7,5
252659,2014-05-03,22:23:00,2013-05-03,NaN,OFF. AGNST PUB ORD SENSBLTY &,AGGRAVATED HARASSMENT 2,MISDEMEANOR,INSIDE,RESIDENCE-HOUSE,QUEENS,40.694933,-73.746375,2014,May,Saturday,0,22,2014-05-03 22:23:00,2014-05-03 22:23:00,5,3,5
335103,2016-09-12,14:00:00,2016-04-12,NaN,HARRASSMENT 2,"HARASSMENT,SUBD 1,CIVILIAN",VIOLATION,INSIDE,RESIDENCE-HOUSE,BRONX,40.858609,-73.908671,2016,Sep,Monday,0,14,2016-09-12 14:00:00,2016-09-12 14:00:00,9,12,0
355423,2010-11-11,07:00:00,2010-11-10,18:00:00,PETIT LARCENY,"LARCENY,PETIT OF LICENSE PLATE",MISDEMEANOR,FRONT OF,STREET,BRONX,40.818753,-73.814761,2010,Nov,Thursday,1,7,2010-11-11 07:00:00,2010-11-11 07:00:00,11,11,3


In [69]:
# drop line where the difference = -366 days, - 365 days, -153 days, -1 day

df_his = df_his.drop(df_his[(diff_h == '-366 days +00:00:00') | (diff_h == '-365 days +00:00:00')|(diff_h == '-153 days +00:00:00')| (diff_h == '-1 days +00:00:00')].index)

In [70]:
# Identifying missing values in offense coloumn

missing_df = df_his[df_his['offense'].isnull()]
print(df_his['offense'].isnull().sum())
display(missing_df.sample(4))

1752


,from_date,from_time,to_date,to_time,offense,offense_description,law_category,occurence,premise,borough,latitude,longitude,year,Month,weekday,holiday,hour,time_str,time,no_of_month,Day_of_month,no_of_weekday
61328,2006-11-21,18:45:00,NaT,NaN,NaN,"CONTEMPT,CRIMINAL",MISDEMEANOR,INSIDE,RESIDENCE - PUBLIC HOUSING,MANHATTAN,40.798483,-73.949147,2006,Nov,Tuesday,0,18,2006-11-21 18:45:00,2006-11-21 18:45:00,11,21,1
65869,2015-10-23,16:40:00,2015-10-23,16:47:00,NaN,SALE OF UNAUTHORIZED RECORDING,MISDEMEANOR,NaN,STREET,BRONX,40.816089,-73.917689,2015,Oct,Friday,0,16,2015-10-23 16:40:00,2015-10-23 16:40:00,10,23,4
96426,2011-07-28,12:05:00,2011-07-28,12:15:00,NaN,FALSE REPORT UNCLASSIFIED,MISDEMEANOR,INSIDE,PUBLIC BUILDING,QUEENS,40.680049,-73.775909,2011,Jul,Thursday,0,12,2011-07-28 12:05:00,2011-07-28 12:05:00,7,28,3
430489,2009-01-14,09:00:00,NaT,NaN,NaN,"THEFT OF SERVICES, UNCLASSIFIE",MISDEMEANOR,INSIDE,RESIDENCE - APT. HOUSE,BRONX,40.830121,-73.919471,2009,Jan,Wednesday,0,9,2009-01-14 09:00:00,2009-01-14 09:00:00,1,14,2


In [71]:
print(len(missing_df.offense_description.unique()))
print(missing_df.offense_description.unique())

25
['RIOT 2/INCITING' 'VIOLATION OF ORDER OF PROTECTI'
 'SALE OF UNAUTHORIZED RECORDING' 'FALSE REPORT UNCLASSIFIED'
 'CONTEMPT,CRIMINAL' 'LEWDNESS,PUBLIC' 'THEFT OF SERVICES, UNCLASSIFIE'
 'NUISANCE,CRIMINAL,UNCLASSIFIED' 'FALSE REPORT BOMB' 'TAX LAW'
 'PUBLIC ADMINISTATION,UNCLASS M' 'DISORDERLY CONDUCT'
 'AGGRAVATED HARASSMENT 2' 'OBSCENITY 1' 'PUBLIC ADMINISTRATION,UNCLASSI'
 'PRIVACY,OFFENSES AGAINST,UNCLA' 'CONTROLLED SUBSTANCE, POSSESSI'
 'NY STATE LAWS,UNCLASSIFIED FEL' 'LARCENY,GRAND FROM PERSON,UNCL'
 'MATERIAL              OFFENSIV' 'MENACING,UNCLASSIFIED'
 'FRAUD,UNCLASSIFIED-MISDEMEANOR' 'CRIMINAL MISCHIEF,UNCLASSIFIED 4'
 'ASSAULT 3' 'CHILD, ENDANGERING WELFARE']


In [72]:
# drop missing values offense column

df_his = df_his[(pd.notnull(df_his['offense']))]
df_his['offense'].isnull().sum()

0

In [73]:
# checking no. of offense type, law category, borough, year

print('no. of offense: ',len(pd.unique(df_his.offense)))
print('no. of law category: ',len(pd.unique(df_his.law_category)))
print('no. of borough: ',len(pd.unique(df_his.borough)))
print('no. of year: ',len(pd.unique(df_his.year)))

no. of offense:  66
no. of law category:  3
no. of borough:  6
no. of year:  11


## Common Criminal Offenses

In [74]:
# Create new dataset from df_2017 & df_his

df_all = pd.concat([df_2017, df_his])

In [75]:
# Top offenses

df_offenses = pd.crosstab(index=df_all["offense"], columns="count")     

df_offenses = df_offenses.sort_values(by='count',ascending=False)
top_offenses = df_offenses.head(30).index.values

In [76]:
# new column for offense category

df_all['offense_cutoff'] = df_all.offense.map(lambda v: v if v in top_offenses else "OTHER")
len(df_all.offense_cutoff.unique())

31

## Save cleaned data into a new csv file

In [77]:
df_all.to_csv('cleaned.csv', index=False)